## 30s k-fold

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Activation
from keras.utils import np_utils
import tensorflow as tf
from keras.callbacks import EarlyStopping
np.set_printoptions(suppress=True)

data = np.load("/home/hsiehch/30s/2D_spect_img/data_RGB.npy")
labels = np.load('/home/hsiehch/30s/2D_spect_img/labels.npy')

data = data / 255
labels = np_utils.to_categorical(labels, 4)

print(data.shape)
print(labels.shape)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(10151, 59, 151, 3)
(10151, 4)


In [2]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
print(kf)

training_data = []
training_label = []
validation_data = []
validation_label = []

for train_index, test_index in kf.split(data):
    print('trian:', train_index, 'len', len(train_index), 'test:', test_index, 'len', len(test_index))
    training_data.append(data[train_index])
    training_label.append(labels[train_index])
    validation_data.append(data[test_index])
    validation_label.append(labels[test_index])

training_data = np.array(training_data)
training_label = np.array(training_label)
validation_data = np.array(validation_data)
validation_label = np.array(validation_label)


KFold(n_splits=5, random_state=None, shuffle=True)
trian: [    0     2     3 ... 10147 10148 10150] len 8120 test: [    1     7     9 ... 10137 10139 10149] len 2031
trian: [    0     1     2 ... 10147 10148 10149] len 8121 test: [    6    10    16 ... 10127 10140 10150] len 2030
trian: [    0     1     3 ... 10147 10149 10150] len 8121 test: [    2     8    17 ... 10142 10146 10148] len 2030
trian: [    1     2     3 ... 10148 10149 10150] len 8121 test: [    0     4    13 ... 10132 10143 10147] len 2030
trian: [    0     1     2 ... 10148 10149 10150] len 8121 test: [    3     5    12 ... 10130 10144 10145] len 2030


In [3]:
training_data[0].shape

(8120, 59, 151, 3)

In [4]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import itertools

def create_model():
    model = Sequential() 
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same', data_format="channels_last", input_shape = (59, 151, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(filters = 64, kernel_size = 3, padding='same'))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Flatten())
    model.add(Dense(160, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation = "relu"))
    model.add(Dense(4, activation = "softmax"))

    return model

def calculate_avg_F1(scores):
    tmp = [0,0,0,0]
    for i in scores:
        tmp += i
    tmp /= 5
    return tmp

def run_model(batch_S, learning_rate):
    
    Scores = []
    for X_train, Y_train, X_val, Y_val in zip(training_data, training_label, validation_data, validation_label):

        model = create_model()
        adam = Adam(lr = learning_rate)
        model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])
        early_stop = EarlyStopping(patience=25)
        train_history_1 = model.fit(x = X_train, 
                                    y = Y_train,
                                    epochs=100,
                                    validation_data=(X_val, Y_val),
                                    callbacks=[early_stop],
                                    batch_size=batch_S, 
                                    verbose=1)
        
        evaluation = model.evaluate(x = X_val, y = Y_val)
        print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evaluation[0], evaluation[1]))

        validation_prediction = model.predict_classes(X_val, batch_size=batch_S)
        score = f1_score(Y_val.argmax(axis=1), validation_prediction, average=None)
        Scores.append(score)
        del model
    
    result = calculate_avg_F1(Scores)
    F1_result.append(result)
    del Scores
    

        
F1_result = []
hyperparameter_index = []
batch_S = [30, 50, 70, 90, 120]
learning_rate = [0.00005, 0.0001, 0.0005, 0.001, 0.005]

for bat, lr in itertools.product(batch_S, learning_rate):
    hyperparameter_index.append([bat, lr])
    run_model(bat, lr)

print('Finish training!')

Train on 8120 samples, validate on 2031 samples
Epoch 1/100
8120/8120 [==============================] - 16s 2ms/step - loss: 1.0889 - acc: 0.5422 - val_loss: 1.0696 - val_acc: 0.5894
Epoch 2/100
8120/8120 [==============================] - 13s 2ms/step - loss: 0.9423 - acc: 0.6025 - val_loss: 1.0104 - val_acc: 0.6425
Epoch 3/100
8120/8120 [==============================] - 12s 2ms/step - loss: 0.8418 - acc: 0.6453 - val_loss: 0.8834 - val_acc: 0.6484
Epoch 4/100
8120/8120 [==============================] - 13s 2ms/step - loss: 0.7970 - acc: 0.6683 - val_loss: 0.8571 - val_acc: 0.6169
Epoch 5/100
8120/8120 [==============================] - 14s 2ms/step - loss: 0.7538 - acc: 0.6857 - val_loss: 0.7872 - val_acc: 0.6814
Epoch 6/100
8120/8120 [==============================] - 14s 2ms/step - loss: 0.7252 - acc: 0.7015 - val_loss: 0.7834 - val_acc: 0.6805
Epoch 7/100
8120/8120 [==============================] - 13s 2ms/step - loss: 0.6970 - acc: 0.7113 - val_loss: 0.7871 - val_acc: 0.6484


8120/8120 [==============================] - 13s 2ms/step - loss: 0.2996 - acc: 0.8846 - val_loss: 0.6024 - val_acc: 0.7696
Epoch 61/100
8120/8120 [==============================] - 12s 2ms/step - loss: 0.2961 - acc: 0.8904 - val_loss: 0.5401 - val_acc: 0.7893
Epoch 62/100
8120/8120 [==============================] - 12s 2ms/step - loss: 0.2829 - acc: 0.8959 - val_loss: 0.6523 - val_acc: 0.7573
Epoch 63/100
8120/8120 [==============================] - 13s 2ms/step - loss: 0.2885 - acc: 0.8903 - val_loss: 0.5288 - val_acc: 0.7952
Epoch 64/100
8120/8120 [==============================] - 12s 2ms/step - loss: 0.2771 - acc: 0.8917 - val_loss: 0.5481 - val_acc: 0.7863
Epoch 65/100
8120/8120 [==============================] - 13s 2ms/step - loss: 0.2675 - acc: 0.8989 - val_loss: 0.5516 - val_acc: 0.7917
Epoch 66/100
8120/8120 [==============================] - 12s 2ms/step - loss: 0.2663 - acc: 0.8984 - val_loss: 0.5735 - val_acc: 0.7809
Epoch 67/100
8120/8120 [==============================

Epoch 25/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.5373 - acc: 0.7840 - val_loss: 0.6287 - val_acc: 0.7631
Epoch 26/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.5302 - acc: 0.7883 - val_loss: 0.6204 - val_acc: 0.7596
Epoch 27/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.5206 - acc: 0.7957 - val_loss: 0.6058 - val_acc: 0.7778
Epoch 28/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.5194 - acc: 0.7962 - val_loss: 0.6112 - val_acc: 0.7759
Epoch 29/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.5114 - acc: 0.7977 - val_loss: 0.6208 - val_acc: 0.7813
Epoch 30/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.5000 - acc: 0.8041 - val_loss: 0.6624 - val_acc: 0.7172
Epoch 31/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.4992 - acc: 0.8048 - val_loss: 0.6464 - val_acc: 0.7685
Epoch 32/100
8121/8121 [=================

Epoch 85/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.1683 - acc: 0.9333 - val_loss: 0.6026 - val_acc: 0.7833
Epoch 86/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.1671 - acc: 0.9350 - val_loss: 0.5871 - val_acc: 0.7926
Epoch 87/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.1592 - acc: 0.9420 - val_loss: 0.7531 - val_acc: 0.7586
Epoch 88/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.1520 - acc: 0.9432 - val_loss: 0.5594 - val_acc: 0.7867
Epoch 89/100
8121/8121 [==============================] - 13s 2ms/step - loss: 0.1483 - acc: 0.9416 - val_loss: 0.6306 - val_acc: 0.7892
Epoch 90/100
8121/8121 [==============================] - 13s 2ms/step - loss: 0.1591 - acc: 0.9397 - val_loss: 0.6631 - val_acc: 0.7655
Epoch 91/100
2030/2030 [==============================] - 1s 634us/step
Loss: 0.605, Accuracy: 0.792
Train on 8121 samples, validate on 2030 samples
Epoch 1/100
8121/8121 [======

Epoch 53/100
8121/8121 [==============================] - 13s 2ms/step - loss: 0.3495 - acc: 0.8681 - val_loss: 0.5340 - val_acc: 0.7852
Epoch 54/100
8121/8121 [==============================] - 13s 2ms/step - loss: 0.3442 - acc: 0.8711 - val_loss: 0.5752 - val_acc: 0.7596
Epoch 55/100
8121/8121 [==============================] - 13s 2ms/step - loss: 0.3361 - acc: 0.8714 - val_loss: 0.5624 - val_acc: 0.7842
Epoch 56/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.3317 - acc: 0.8749 - val_loss: 0.5333 - val_acc: 0.7862
Epoch 57/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.3355 - acc: 0.8737 - val_loss: 0.5578 - val_acc: 0.7828
Epoch 58/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.3252 - acc: 0.8780 - val_loss: 0.5442 - val_acc: 0.7793
Epoch 59/100
8121/8121 [==============================] - 14s 2ms/step - loss: 0.3097 - acc: 0.8835 - val_loss: 0.5656 - val_acc: 0.7749
Epoch 60/100
8121/8121 [=================

...

Epoch 40/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5124 - acc: 0.8045 - val_loss: 0.7213 - val_acc: 0.7340
Epoch 41/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5082 - acc: 0.8029 - val_loss: 0.6340 - val_acc: 0.7557
Epoch 42/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.4925 - acc: 0.8148 - val_loss: 0.6281 - val_acc: 0.7488
Epoch 43/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.4988 - acc: 0.8111 - val_loss: 0.6736 - val_acc: 0.7502
Epoch 44/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.4829 - acc: 0.8114 - val_loss: 0.6146 - val_acc: 0.7586
Epoch 45/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.4797 - acc: 0.8202 - val_loss: 0.6457 - val_acc: 0.7606
Epoch 46/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.4714 - acc: 0.8200 - val_loss: 0.6020 - val_acc: 0.7788
Epoch 47/100
8121/8121 [=================

Epoch 8/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7790 - acc: 0.6787 - val_loss: 0.8564 - val_acc: 0.6621
Epoch 9/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.7513 - acc: 0.6854 - val_loss: 0.7916 - val_acc: 0.6833
Epoch 10/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7246 - acc: 0.6957 - val_loss: 0.7714 - val_acc: 0.6887
Epoch 11/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.7051 - acc: 0.7095 - val_loss: 0.8367 - val_acc: 0.6759
Epoch 12/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.7063 - acc: 0.7010 - val_loss: 0.7508 - val_acc: 0.7025
Epoch 13/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.6904 - acc: 0.7112 - val_loss: 0.9903 - val_acc: 0.6025
Epoch 14/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.6656 - acc: 0.7277 - val_loss: 0.8611 - val_acc: 0.6502
Epoch 15/100
8121/8121 [===================

Epoch 68/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3882 - acc: 0.8504 - val_loss: 0.5838 - val_acc: 0.7680
Epoch 69/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3726 - acc: 0.8567 - val_loss: 0.6199 - val_acc: 0.7596
Epoch 70/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3775 - acc: 0.8581 - val_loss: 0.5666 - val_acc: 0.7842
Epoch 71/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3609 - acc: 0.8644 - val_loss: 0.5606 - val_acc: 0.7818
Epoch 72/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3587 - acc: 0.8634 - val_loss: 0.5325 - val_acc: 0.7818
Epoch 73/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3489 - acc: 0.8708 - val_loss: 0.5988 - val_acc: 0.7675
Epoch 74/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3517 - acc: 0.8652 - val_loss: 0.5724 - val_acc: 0.7837
Epoch 75/100
8121/8121 [=================

Epoch 30/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5910 - acc: 0.7638 - val_loss: 0.7397 - val_acc: 0.7044
Epoch 31/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5914 - acc: 0.7648 - val_loss: 0.6818 - val_acc: 0.7182
Epoch 32/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5861 - acc: 0.7675 - val_loss: 0.7466 - val_acc: 0.7000
Epoch 33/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5776 - acc: 0.7732 - val_loss: 0.7301 - val_acc: 0.7266
Epoch 34/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5575 - acc: 0.7745 - val_loss: 0.8493 - val_acc: 0.6729
Epoch 35/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5770 - acc: 0.7765 - val_loss: 0.6690 - val_acc: 0.7355
Epoch 36/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.5709 - acc: 0.7736 - val_loss: 0.7455 - val_acc: 0.7133
Epoch 37/100
8121/8121 [=================

Epoch 90/100
8121/8121 [==============================] - 12s 1ms/step - loss: 0.3936 - acc: 0.8510 - val_loss: 0.5707 - val_acc: 0.7823
Epoch 91/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3796 - acc: 0.8612 - val_loss: 0.5695 - val_acc: 0.7837
Epoch 92/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3624 - acc: 0.8638 - val_loss: 0.6149 - val_acc: 0.7591
Epoch 93/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3644 - acc: 0.8621 - val_loss: 0.5492 - val_acc: 0.7926
Epoch 94/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3599 - acc: 0.8674 - val_loss: 0.6132 - val_acc: 0.7675
Epoch 95/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3589 - acc: 0.8665 - val_loss: 0.6117 - val_acc: 0.7635
Epoch 96/100
8121/8121 [==============================] - 11s 1ms/step - loss: 0.3741 - acc: 0.8600 - val_loss: 0.5394 - val_acc: 0.7985
Epoch 97/100
8121/8121 [=================

In [5]:
for bat, lr in itertools.product(batch_S, learning_rate):
    print(bat, lr)

30 5e-05
30 0.0001
30 0.0005
30 0.001
30 0.005
50 5e-05
50 0.0001
50 0.0005
50 0.001
50 0.005
70 5e-05
70 0.0001
70 0.0005
70 0.001
70 0.005
90 5e-05
90 0.0001
90 0.0005
90 0.001
90 0.005
120 5e-05
120 0.0001
120 0.0005
120 0.001
120 0.005


In [6]:
import pprint
pprint.pprint(F1_result)

[array([0.66834055, 0.52476712, 0.87090963, 0.66587572]),
 array([0.65508542, 0.5485832 , 0.87234579, 0.66106634]),
 array([0.63620587, 0.53174942, 0.86806507, 0.67333027]),
 array([0.68673129, 0.51869788, 0.87052678, 0.6696205 ]),
 array([0.49913675, 0.26887115, 0.82295756, 0.45180504]),
 array([0.67044206, 0.55287008, 0.86807528, 0.65768723]),
 array([0.67268886, 0.54829908, 0.86707249, 0.66812725]),
 array([0.65772061, 0.4988153 , 0.86604544, 0.67691145]),
 array([0.65391337, 0.51712745, 0.87446289, 0.65770701]),
 array([0.6401902 , 0.51063389, 0.859286  , 0.59201119]),
 array([0.56211778, 0.55043751, 0.87667902, 0.67489162]),
 array([0.60862027, 0.47811457, 0.87257359, 0.66223681]),
 array([0.67320131, 0.49829942, 0.87105604, 0.65684055]),
 array([0.64184123, 0.53007974, 0.86503449, 0.65579779]),
 array([0.49163775, 0.35242272, 0.84897119, 0.56899653]),
 array([0.65044999, 0.56037346, 0.87192674, 0.65552668]),
 array([0.62718727, 0.51916667, 0.8638596 , 0.63734379]),
 array([0.6715